In [1]:
import numpy as np
import torch
import faiss
import faiss.contrib.torch_utils

In [2]:
d = 2048                         # dimension
nb = 400_000                      # database size
# nq = 100_000                       # nb of queries
np.random.seed(1234)             # make reproducible

# xb = np.random.random((nb, d)).astype('float32')
# xb[:, 0] += np.arange(nb) / 1000.
# xq = np.random.random((nq, d)).astype('float32')
# xq[:, 0] += np.arange(nq) / 1000.

xb = torch.rand((nb, d), device='cuda:0', dtype=torch.float32)
xb[:, 0] += torch.arange(nb, device='cuda:0') / 1000.
# xq = torch.rand((nq, d), device='cuda:0', dtype=torch.float32)
# xq[:, 0] += torch.arange(nq, device='cuda:0') / 1000.

In [3]:
res = faiss.StandardGpuResources()  # use a single GPU

In [4]:
nlist = 100
# build a flat (CPU) index
gpu_index_flat = faiss.GpuIndexFlatL2(res, d)
# make it into a gpu index
# gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)

In [6]:
gpu_index_flat.add(xb)         # add vectors to the index
print(gpu_index_flat.ntotal)

200000


/home/yasin/anaconda3/envs/py310/lib/python3.10/site-packages/faiss/contrib/torch_utils.py:51: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage().data_ptr() + x.storage_offset() * 4)


In [7]:
k = 10                          # we want to see 4 nearest neighbors
D, I = gpu_index_flat.search(xq, k)  # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

tensor([[ 706, 1171,  975, 1680, 1864, 1375, 1644,  335, 2007,  191],
        [ 651,  215, 1532, 1651,  147, 1146,  892,  371,  436,  400],
        [1285,  300, 2695, 1143, 1934, 3096, 1026,  836,  664, 1776],
        [ 906,  202, 2965,  910,  604, 1192, 2006, 2341, 2429,  968],
        [1092, 3209, 1024, 2283, 1284,  191,   96,  659, 1306,  195]],
       device='cuda:0')
tensor([[ 98086,  99700, 100283,  99983,  97837, 100412,  98513, 100179,  97918,
          98974],
        [100671, 100348, 100330,  98867,  98239, 100205, 100281, 100312,  99379,
          98484],
        [ 99813, 101268, 101288, 100231,  98875, 101913,  98276,  99230,  99945,
         101175],
        [100014,  98072,  98099,  99725, 101310, 100694,  99239, 100072,  99165,
         100861],
        [101577,  99241,  99771, 100348, 100301, 101323,  98939, 100738, 101209,
         100453]], device='cuda:0')
